In [1]:
import schedule
import time
from gtts import gTTS
import os
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')

In [2]:
import winsound
import numpy as np
import pandas as pd
import math
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
from datetime import datetime as dt

In [3]:
from threading import Thread
import json
from scipy.signal import argrelextrema
import pytz
import itertools
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML

In [4]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [5]:
if not mt5.initialize(path="C:/JoeC/MetaTrader 5/terminal64.exe",login=1045610, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [13]:
start = dt.now()

Srates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_H8, 0, 600)

S = pd.DataFrame(Srates)
S['time']=pd.to_datetime(S['time'], unit='s')
S = S.drop(columns=['tick_volume', 'spread', 'real_volume'])
S = S.set_index('time')
S = S.tz_localize('Etc/GMT-7')
S = S.tz_convert('UTC')

S = S.assign(MOVE=(S['close'] - S['open']))
SAvgCandle = np.average(abs(S['MOVE']))
STDCandle = np.std(abs(S['MOVE']))
CP = S['close'].iloc[-1]
scope = SAvgCandle*15
SCM = SAvgCandle

CNDL = (S.iloc[:, 3]) - (S.iloc[:, 0])
S['CNDL'] = np.where(CNDL > 0, 1, -1)
S = S.assign(INFL=S['CNDL'].ne(S['CNDL'].shift()))
S = S[S.INFL != 0]
S['INFL'] = S['CNDL']*-1
S['LVL'] = np.where(S.INFL == 1, S.high, S.low)
S = S.assign(SIZE=abs(S['LVL'] - (S['LVL'].shift(-1))))

A = np.array(S.LVL)

B = list(combinations(A, 2))
Ba = np.array([x for x, y in B])
B = np.array([abs(x - y) for x, y in B])
Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])

T = list(zip(Ba,Bb))

table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
table = table.sort_values(['a'], ascending=False)
table = table.rename(columns={'b': 'Pings'})
table = table.fillna(0)
table = table[table['max'] != 0]
pingsavg = np.average(table['Pings'])
table = table.assign(level=table.index)

levels = list(table['level'])

end = dt.now()
elapsed = end - start
elapsed = elapsed.total_seconds()
elapsed = str(elapsed)
timer = elapsed + '   seconds'
print(timer)
print(levels)

0.276511   seconds
[1.13547, 1.12391, 1.12057, 1.11859, 1.11755, 1.11584, 1.11566, 1.11541, 1.11516, 1.11497, 1.11443, 1.11398, 1.11338, 1.1131, 1.11249, 1.11226, 1.11159, 1.11147, 1.11127, 1.11091, 1.1105, 1.11035, 1.1102400000000001, 1.11022, 1.1098, 1.10943, 1.10897, 1.10873, 1.10858, 1.10842, 1.10801, 1.10775, 1.10768, 1.10754, 1.10735, 1.10698, 1.1068500000000001, 1.10675, 1.10658, 1.10581, 1.10534, 1.10529, 1.10426, 1.10398, 1.10391, 1.10386, 1.10353, 1.10298, 1.10264, 1.10254, 1.10229, 1.10224, 1.10172, 1.10166, 1.10156, 1.10107, 1.10065, 1.10042, 1.10009, 1.09974, 1.09968, 1.09928, 1.09913, 1.09852, 1.0979700000000001, 1.09686, 1.09551, 1.09461, 1.09335, 1.09204, 1.09078, 1.09049, 1.09016, 1.08879, 1.08791, 1.08733, 1.0867499999999999, 1.08611, 1.08592, 1.0855, 1.08525, 1.08491, 1.08453, 1.08418, 1.08324, 1.0832, 1.0830899999999999, 1.08208, 1.08166, 1.08153, 1.08141, 1.08079, 1.08019, 1.07903, 1.07822, 1.07781, 1.0760399999999999]


In [9]:
#mt5.shutdown()